In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
data = pd.read_csv("train.csv")
data1 = pd.read_csv("store.csv")

/tmp/ipython-input-2639253382.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("train.csv")


In [19]:
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [20]:
data1.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [21]:
data_final = data.merge(data1, on='Store', how='left')

In [22]:
data_final.isnull().sum()

,0
Store,0
DayOfWeek,0
Date,0
Sales,0
Customers,0
Open,0
Promo,0
StateHoliday,0
SchoolHoliday,0
StoreType,0


In [23]:
columns_to_drop = ['Promo2SinceWeek','Promo2SinceYear','PromoInterval']
data_final = data_final.drop(columns_to_drop,axis =1 )

In [24]:
data_final['CompetitionOpenSinceMonth'] = data_final['CompetitionOpenSinceMonth'].fillna(np.mean(data_final['CompetitionOpenSinceMonth']))
data_final['CompetitionOpenSinceYear'] = data_final['CompetitionOpenSinceYear'].fillna(np.mean(data_final['CompetitionOpenSinceYear']))
data_final['CompetitionDistance'] = data_final['CompetitionDistance'].fillna(np.mean(data_final['CompetitionDistance']))
data_final['StateHoliday'] = (data_final['StateHoliday'] == '0').astype(int)
data_final = data_final.dropna()

In [25]:
data_final.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2
0,1,5,2015-07-31,5263,555,1,1,1,1,c,a,1270.0,9.0,2008.0,0
1,2,5,2015-07-31,6064,625,1,1,1,1,a,a,570.0,11.0,2007.0,1
2,3,5,2015-07-31,8314,821,1,1,1,1,a,a,14130.0,12.0,2006.0,1
3,4,5,2015-07-31,13995,1498,1,1,1,1,c,c,620.0,9.0,2009.0,0
4,5,5,2015-07-31,4822,559,1,1,1,1,a,a,29910.0,4.0,2015.0,0


In [26]:
from sklearn.preprocessing import StandardScaler

column_to_normalize = ['Sales',"Customers","CompetitionDistance"	,"CompetitionOpenSinceMonth"	,"CompetitionOpenSinceYear"]
def normalize_column(column):
  scaler = StandardScaler()
  normalized_column = scaler.fit_transform(data_final[[column]])
  data_final[column] = normalized_column
for column in column_to_normalize:
  normalize_column(column)


In [27]:
columns_to_one_hot = ['StoreType','Assortment']

for column in columns_to_one_hot:
  # Get dummy variables with a prefix to avoid name clashes
  dummies = pd.get_dummies(data_final[column], prefix=column, drop_first=True)
  # Concatenate the dummy variables with the original DataFrame
  data_final = pd.concat([data_final, dummies], axis=1)
  # Drop the original column
  data_final = data_final.drop(column, axis=1)

In [28]:
data_final

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c
0,1,5,2015-07-31,-0.132683,-0.168269,1,1,1,1,-0.539900,6.699409e-01,-1.394579e-01,0,False,True,False,False,False
1,2,5,2015-07-31,0.075373,-0.017540,1,1,1,1,-0.630746,1.423898e+00,-3.415040e-01,1,False,False,False,False,False
2,3,5,2015-07-31,0.659800,0.404499,1,1,1,1,1.129083,1.800876e+00,-5.435502e-01,1,False,False,False,False,False
3,4,5,2015-07-31,2.135414,1.862258,1,1,1,1,-0.624257,6.699409e-01,6.258834e-02,0,False,True,False,False,True
4,5,5,2015-07-31,-0.247231,-0.159656,1,1,1,1,3.177025,-1.214950e+00,1.274865e+00,0,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,-1.499723,-1.363330,0,0,0,1,-0.458138,-4.609939e-01,1.072819e+00,1,False,False,False,False,False
1017205,1112,2,2013-01-01,-1.499723,-1.363330,0,0,0,1,-0.460733,-1.214950e+00,-5.435502e-01,0,False,True,False,False,True
1017206,1113,2,2013-01-01,-1.499723,-1.363330,0,0,0,1,0.497050,-1.339296e-15,1.837599e-13,0,False,False,False,False,True
1017207,1114,2,2013-01-01,-1.499723,-1.363330,0,0,0,1,-0.591812,-1.339296e-15,1.837599e-13,0,False,False,False,False,True


In [30]:
from xgboost import XGBRegressor

# Create lag features (use previous days' sales to predict today)
data_final = data_final.sort_values(['Store', 'Date'])

data_final['Sales_lag1'] = data_final.groupby('Store')['Sales'].shift(1)
data_final['Sales_lag7'] = data_final.groupby('Store')['Sales'].shift(7)  # Last week
data_final['Sales_lag30'] = data_final.groupby('Store')['Sales'].shift(30)  # Last month

# Drop rows with NaN from lag features
data_final = data_final.dropna()

# Drop the 'Date' column as it's an object type and not directly usable by XGBoost
X = data_final.drop(['Sales', 'Date'], axis=1)
Y = data_final['Sales']

# Train/test split (time-based, not random!)
split_point = int(len(X) * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = Y[:split_point], Y[split_point:]

# Train
model = XGBRegressor()
model.fit(X_train, y_train)

# Predict
result = model.predict(X_test)

# Evaluate (use RMSE or MAE, not accuracy for regression!)
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, result))}")
print(f"MAE: {mean_absolute_error(y_test, result)}")

RMSE: 0.2609505230444135
MAE: 0.17592617843482802
